<a href="https://colab.research.google.com/github/pablo-arantes/making-it-rain/blob/main/Partial_Charges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hello there!**

This is a Jupyter notebook for calculating partial charges using Psi4. Psi4 is an open-source suite of ab initio quantum chemistry programs designed for efficient, high-accuracy simulations of molecular properties. 
This notebook is a supplementary material of the paper "***Making it rain: Cloud-based molecular simulations for everyone***" ([link here](https://doi.org/10.1021/acs.jcim.1c00998)) and we encourage you to read it before using this pipeline.

--- 
**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/pablo-arantes/making-it-rain/issues

**Acknowledgments**
- We would like to thank the [Psi4](https://psicode.org/) team for developing an excellent and open source suite of ab initio quantum chemistry.

- Thank you to Doctor [Souvik Sinha](https://twitter.com/sou_svk) for his help in the code process.

- Making-it-rain by **Pablo R. Arantes** ([@pablitoarantes](https://twitter.com/pablitoarantes)), **Marcelo D. Polêto** ([@mdpoleto](https://twitter.com/mdpoleto)), **Conrado Pedebos** ([@ConradoPedebos](https://twitter.com/ConradoPedebos)) and **Rodrigo Ligabue-Braun** ([@ligabue_braun](https://twitter.com/ligabue_braun)).


- Also, credit to [David Koes](https://github.com/dkoes) for his awesome [py3Dmol](https://3dmol.csb.pitt.edu/) plugin.

- Thank you to [Jonas Boström](https://twitter.com/DrBostrom) for his **Psi4** code. If you use the notebook, please, [cite](https://pubs.acs.org/doi/10.1021/acs.jcim.1c01535).

- For related notebooks see: [Making-it-rain](https://github.com/pablo-arantes/making-it-rain)

# **Introduction**

In this notebook, we will calculate the partial charges of Ibuprofen using [Psi4](https://psicode.org/). It takes a SMILES and outputs a mol2 file with RESP derived partial charges. Options for setting method (HF, B3LYP, ...), basis set (3-21G*, 6-31G**) and singlepoint or geometry optimization are available. 

In terms of inputs, we wil need:
*  A SMILES of your molecule.

The input SMILES example for Ibuprofen: CC(C)CC1=CC=C(C=C1)C(C)C(=O)O 

## ---







---
# **Setting the environment for QM calculation**

Firstly, we need to install all necessary libraries and packages for our calculation. The main packages we will be installing are:

1.    Anaconda (https://docs.conda.io/en/latest/miniconda.html)
2.    py3Dmol (https://pypi.org/project/py3Dmol/)
3.    Matplotlib (https://matplotlib.org/)
4.    Psi4 (https://psicode.org/)
5.    RDKit (https://www.rdkit.org/)

In [ ]:
%%capture
%%bash

#@title **Install dependencies**
#@markdown It will take a few minutes, please, drink a coffee and wait. ;-)
# install dependencies
# %%capture

# install conda
pip -q install py3Dmol 2>&1 1>/dev/null
wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh 
bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
rm -r Miniconda3-latest-Linux-x86_64.sh
conda create -n p4env python=3.9 psi4 resp -c psi4/label/dev --yes
source activate p4env
pip install rdkit-pypi
pip install Cython
# conda install -c psi4 resp --yes
conda install -c conda-forge parmed  --yes 2>&1 1>/dev/null
conda install -c bioconda pybel --yes
conda install -c conda-forge openbabel --yes
conda install -c anaconda ipython

python
import os
import psi4
import resp
import openbabel as ob
from rdkit import Chem
from rdkit.Chem import AllChem


#load dependencies
# sys.path.append('/usr/local/lib/python3.7/site-packages/')
# import parmed as pmd
# import os
# import urllib.request  
# import numpy as np
# import py3Dmol
# from __future__ import print_function
# import platform
# import scipy.cluster.hierarchy
# from scipy.spatial.distance import squareform
# import scipy.stats as stats
# import matplotlib.pyplot as plt
# import pandas as pd
# from scipy.interpolate import griddata
# import seaborn as sb
# from statistics import mean, stdev
# from pytraj import matrix
# from matplotlib import colors
# from IPython.display import set_matplotlib_formats


---
# **Loading the necessary input files**

At this point, we should have all libraries and dependencies installed.

Below, you should provide the smiles of your molecule.


In [ ]:
%%bash
source activate p4env

python

import sys
import os
import psi4
import resp
import rdkit
from openbabel import openbabel as ob
from rdkit import Chem
from rdkit.Chem import AllChem,Draw
import IPython
from IPython.display import Image
# from rdkit.Chem.Draw import IPythonConsole

#@title **Please, provide the necessary input files below**: 

#@markdown Type the smiles of your molecule. If you don't know your smiles, please, check at https://pubchem.ncbi.nlm.nih.gov/
Smiles = 'CC(C)CC1=CC=C(C=C1)C(C)C(=O)O' #@param {type:"string"}
smiles_fig = Chem.MolFromSmiles(Smiles)



# Draw.MolToMPL(smiles_fig, size=(200, 200))
smi_end = Draw.MolToFile(smiles_fig, size=(600, 600), filename='smiles.png')
Image('smiles.png')

#print("SMILES:", Smiles)

original_stdout = sys.stdout # Save a reference to the original standard output
final_smiles = '/content/smiles.smi'
with open(final_smiles, 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(Smiles, ' ')
    sys.stdout = original_stdout

In [ ]:
import sys
import os
import IPython
from IPython.display import Image
# from rdkit.Chem.Draw import IPythonConsole

#@title **Check your molecule**: 

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('smiles.png')
plt.figure(figsize = (8,8))
imgplot = plt.imshow(img)
plt.axis('off')
plt.show()

# Image('smiles.png')
a_file = open('/content/smiles.smi')
file_contents = a_file.read()
print("SMILES:", file_contents)
#print("SMILES:", Smiles)


In [ ]:
%%bash
source activate p4env

python
import os
import psi4
import resp
from openbabel import openbabel as ob
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem,Draw
IPythonConsole.molSize = 250,250

def neutralize_atoms(mol):
    pattern = Chem.MolFromSmarts("[+1!h0!$([*]~[-1,-2,-3,-4]),-1!$([*]~[+1,+2,+3,+4])]")
    at_matches = mol.GetSubstructMatches(pattern)
    at_matches_list = [y[0] for y in at_matches]
    if len(at_matches_list) > 0:
        for at_idx in at_matches_list:
            atom = mol.GetAtomWithIdx(at_idx)
            chg = atom.GetFormalCharge()
            hcount = atom.GetTotalNumHs()
            atom.SetFormalCharge(0)
            atom.SetNumExplicitHs(hcount - chg)
            atom.UpdatePropertyCache()
    return mol

def cleanUp(psi4out_xyz):
    deleteTheseFiles = ['1_default_grid.dat','1_default_grid_esp.dat','grid.dat','timer.dat']
    deleteTheseFiles.append(psi4out_xyz)
    for fileName in deleteTheseFiles:
        if os.path.exists(fileName):
            os.remove(fileName)

def get_xyz_coords(mol):
    if not mol is None:
        num_atoms = mol.GetNumAtoms()
        xyz_string=""
        for counter in range(num_atoms):
            pos=mol.GetConformer().GetAtomPosition(counter)
            xyz_string = xyz_string + ("%s %12.6f %12.6f %12.6f\n" % (mol.GetAtomWithIdx(counter).GetSymbol(), pos.x, pos.y, pos.z) )
    return xyz_string


def calcRESPCharges(mol, basisSet, method, gridPsi4 = 1):
    options = {'BASIS_ESP': basisSet,
               'METHOD_ESP': method,
               'RESP_A': 0.0005,
               'RESP_B': 0.1,
               'VDW_SCALE_FACTORS':[1.4, 1.6, 1.8, 2.0],
               'VDW_POINT_DENSITY':int(gridPsi4)
    }

    # resp_charges = resp.resp([mol], [options])[0][1]
    resp_charges = resp.resp([mol], options)
    
    return resp_charges

#@title **Parameters to generate partial charges:**


method = "B3LYP" #@param ["B3LYP", "MP2", "HF"]

basisSet = "6-31G**" #@param ["STO-3g", "3-21G", "6-31G*", "6-31G**", "6-31+G(d,p)", "6-311++G(2d,p)"]

Neutralize = "Yes" #@param ["Yes", "No"]

if Neutralize == "Yes":
  neutralize = True
else:
  neutralize = False

Calculation_Type = "Single Point" #@param ["Single Point", "Geometry Optimization"]

if Calculation_Type == "Single Point":
  singlePoint = True
else:
  singlePoint = False  

# method = 'b3lyp'
# basisSet = '6-31G**' #'6-31G**' '3-21g'
neutralize = True
psi4.set_memory('8 GB')
obConversion = ob.OBConversion()
obConversion.SetInAndOutFormats("xyz", "mol2")
# singlePoint = True
path = '/content'

# inputFile = '/content/smiles.smi'
# molList = Chem.SDMolSupplier(inputFile, removeHs=False)

SMILESasInput = True

if SMILESasInput:
    molList = []
    inputFile = '/content/smiles.smi'
    suppl = Chem.SmilesMolSupplier(inputFile, titleLine = False)

    for mol in suppl:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        try: 
            AllChem.MMFFOptimizeMolecule(mol)
        except: 
            AllChem.UFFOptimizeMolecule(mol)            
        molList.append(mol)

for mol in molList:
    print(mol)
    if not mol is None:

        molId = mol.GetProp("_Name")
        print('Trying:', molId)
        
        if neutralize:
            mol = neutralize_atoms(mol)
            mol = Chem.AddHs(mol)

        xyz_string = get_xyz_coords(mol) 
        psi_mol = psi4.geometry(xyz_string)

        ### single point calculation
        outfile_mol2 = inputFile[:-4]+".mol2"

        if singlePoint:
            print('Running singlepoint...')
            resp_charges = calcRESPCharges(psi_mol, basisSet, method, gridPsi4 = 1)

        else:
            print('Running geometry optimization...')            
            methodNbasisSet = method+"/"+basisSet
            psi4.optimize(methodNbasisSet, molecule=psi_mol)
            resp_charges = calcRESPCharges(psi_mol, basisSet, method, gridPsi4 = 1)            

        ### save coords to xyz file 
        psi4out_xyz = molId + '.xyz'
        psi_mol.save_xyz_file(psi4out_xyz,1)


        ### read xyz file and write as mol2 
        ob_mol = ob.OBMol()
        obConversion.ReadFile(ob_mol, psi4out_xyz)

         ### write as mol2 
        outfile_mol2 = path+"/"+molId+"_partialChgs.mol2"
        obConversion.WriteFile(ob_mol, outfile_mol2)

        ### set new partial charges
        count = 0
        newChg_temp = resp_charges[1]
        print("RESP Charges: ", newChg_temp)
        for atom in ob.OBMolAtomIter(ob_mol):
            newChg = newChg_temp[count]
            atom.SetPartialCharge(newChg)
            count += 1

        ### write as mol2 
        outfile_mol2 = path+"/"+molId+"_partialChgs.mol2"
        outfile_pdb = path+"/"+molId+".pdb"
        print("Finished. Saved compound with partial charges as mol2 file: %s" % outfile_mol2)
        obConversion.WriteFile(ob_mol, outfile_mol2)
        ## clean up
        cleanUp(psi4out_xyz)

#draw_with_charges
inputFile2 = '/content/smiles.smi'
suppl2 = Chem.SmilesMolSupplier(inputFile2, titleLine = False)
for mol_2 in suppl2:
    m2 = mol_2

for at, i in zip(m2.GetAtoms(), newChg_temp):
    lbl = '%.2f'%(i)
    at.SetProp('atomNote',lbl)
    Draw.MolToFile(m2, size=(600, 600), filename='smiles_charges.png')

In [ ]:
import sys
import os
import IPython
from IPython.display import Image
# from rdkit.Chem.Draw import IPythonConsole

#@title **Check your charges**: 

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('smiles_charges.png')
plt.figure(figsize = (8,8))
imgplot = plt.imshow(img)
plt.axis('off')
plt.show()

# Image('smiles.png')
a_file = open('/content/smiles.smi')
file_contents = a_file.read()
print("SMILES:", file_contents)

In [ ]:
#@title **Check your geometry**
import warnings
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages/')
import py3Dmol

warnings.filterwarnings('ignore')

def show_pdb():
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open('/content/_partialChgs.mol2','r').read(),'mol2')
  view.addStyle({'stick':{'colorscheme':'greenCarbon','radius':0.3}})
  view.setViewStyle({'style':'outline','color':'black','width':0.1})

  view.zoomTo()
  return view

show_pdb().show()

In [ ]:
#@title **Download your results**
from google.colab import files
!zip -FSr "results.zip" *".smi" *".mol2" *".out" *".png" >/dev/null 2>&1
files.download('results.zip')